In [ ]:
pip install scikit-learn==1.6.0

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
import pandas as pd

df = pd.read_csv('employee_bonus_data_cleaned.csv')

X = df.drop('Премия', axis=1)
y = df['Премия']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return y_pred, mae, mse, r2

models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "SVR": SVR(),
    "XGBoost": XGBRegressor(random_state=42)
}

rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
rf_grid = GridSearchCV(RandomForestRegressor(random_state=42), rf_params, cv=3, scoring='r2')
models["Random Forest (Tuned)"] = rf_grid

gb_params = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5]
}
gb_grid = GridSearchCV(GradientBoostingRegressor(random_state=42), gb_params, cv=3, scoring='r2')
models["Gradient Boosting (Tuned)"] = gb_grid

results = {}
for name, model in models.items():
    print(f"Обучение модели: {name}")
    y_pred, mae, mse, r2 = evaluate_model(model, X_train, X_test, y_train, y_test)
    results[name] = {"MAE": mae, "MSE": mse, "R²": r2, "Predictions": y_pred}
    print(f"  MAE: {mae:.2f}")
    print(f"  MSE: {mse:.2f}")
    print(f"  R²: {r2:.2f}")
    print()

best_model = max(results, key=lambda x: results[x]['R²'])
print(f"Лучшая модель: {best_model} с R² = {results[best_model]['R²']:.2f}")

print("\nПримеры предсказаний и реальных значений для лучшей модели:")
best_predictions = results[best_model]['Predictions']
for i in range(10):
    print(f"Реальное значение: {y_test.iloc[i]}, Предсказанное значение: {best_predictions[i]:.2f}")

best_model_name = max(results, key=lambda x: results[x]['R²'])
best_model = models[best_model_name]  # Получаем объект лучшей модели
joblib.dump(best_model, 'best_model.pkl')  # Сохраняем модель в файл
print(f"Лучшая модель '{best_model_name}' сохранена как 'best_model.pkl'.")


loaded_model = joblib.load('best_model.pkl')

new_data = pd.DataFrame({
    'Оценка выполненной работы руководителем': [8.5, 7.0],
    'Факт выполнения плана (%)': [95, 80],
    'Число продаж страхований авто': [12, 5],
    'Число продаж страхований имущества': [8, 3],
    'Число дней больничного': [2, 4],
    'Число дней отпуска': [5, 7]
})

predictions = loaded_model.predict(new_data)
print("\nПредсказанные премии для новых данных:")
for i, pred in enumerate(predictions):
    print(f"Сотрудник {i + 1}: {pred:.2f} руб.")

Обучение модели: Linear Regression
  MAE: 3135.72
  MSE: 22353190.45
  R²: 0.96

Обучение модели: Random Forest
  MAE: 4773.66
  MSE: 35385262.26
  R²: 0.94

Обучение модели: Gradient Boosting
  MAE: 3548.18
  MSE: 20093150.76
  R²: 0.97

Обучение модели: SVR
  MAE: 19793.49
  MSE: 573252401.55
  R²: 0.01

Обучение модели: XGBoost
  MAE: 3587.99
  MSE: 21846898.73
  R²: 0.96

Обучение модели: Random Forest (Tuned)
  MAE: 4745.50
  MSE: 35336077.41
  R²: 0.94

Обучение модели: Gradient Boosting (Tuned)
  MAE: 2871.61
  MSE: 13671339.99
  R²: 0.98

Лучшая модель: Gradient Boosting (Tuned) с R² = 0.98

Примеры предсказаний и реальных значений для лучшей модели:
Реальное значение: 31200.0, Предсказанное значение: 31231.99
Реальное значение: 10000.0, Предсказанное значение: 15316.51
Реальное значение: 48300.0, Предсказанное значение: 45193.14
Реальное значение: 59000.0, Предсказанное значение: 56597.71
Реальное значение: 42800.0, Предсказанное значение: 48041.64
Реальное значение: 40500.0, 